<a href="https://colab.research.google.com/github/skyloma/img2/blob/master/TPU_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 加载google云盘

In [0]:
from google.colab import drive
drive.mount('/content/drive')


# 切换目录


In [0]:
import os
os.chdir("drive/My Drive/models_pk_rmb/models/research")


In [0]:
!ls

# 授权访问google cloud partform

In [0]:
from google.colab import auth
auth.authenticate_user()
project_id = 'dreamer8090'
!gcloud config set project {project_id}

In [0]:
!gcloud auth application-default login

# 复制文件到google storage

In [0]:
bucket_name="gs://staging.dreamer8090.appspot.com"
# !gsutil cp -r /content/drive/My\ Drive/ssd_mobilenet_v1_ppn_shared_box_predictor_300x300_coco14_sync_2018_07_03/ gs://{bucket_name}/data
!gsutil cp -r /content/drive/My\ Drive/models_pk_rmb/models/research/data {bucket_name}
!gsutil cp test.record  {bucket_name}/data/test.record  
!gsutil cp train.record  {bucket_name}/data/train.record  
# !gsutil cp pk_rmb.pbtxt gs://{bucket_name}/data/pk_rmb.pbtxt    


# 切换到目录research下进行训练

In [0]:
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='gs://staging.dreamer8090.appspot.com/tpu128')

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://29da42d8.ngrok.io


In [33]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
--gcp_project=dreamer8090  \
--pipeline_config_path=tpu128.config \
--model_dir=gs://staging.dreamer8090.appspot.com/tpu128 \
--num_train_steps=100000 \
--num_eval_steps=8000

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1031 09:06:10.266796 140516035643264 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1031 09:06:10.273716 140516035643264 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W1031 09:06:10.273996 140516035643264 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W1031 09:06:10.274173 140516035643264 module_wrapper.py:139] From /content/models/res

In [0]:

# !export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
# --logtostderr \
# --pipeline_config_path=/content/drive/My Drive/tpu128.config \
# --model_dir=gs://mytf9133.appspot.com/train128 \
# --num_train_steps=200000 \
# --num_eval_steps=8000

In [0]:
# !export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_main.py \
# --logtostderr \
# --pipeline_config_path=data/gpu128.config \
# --model_dir=gpu128 \
# --num_train_steps=50000 \
# --num_eval_steps=8000

# 导出训练生成的文件

In [0]:
bucket_name="gs://staging.dreamer8090.appspot.com"
!gsutil cp  -r gs://staging.dreamer8090.appspot.com/tpu128  outdata/
  
# !gsutil cp  gs://{bucket_name}/data/pk_rmb.pbtxt A/
  
# !gsutil cp  gs://{bucket_name}/A/tflite_graph.pbtxt A/  

# 导出能转换成tflte的pb

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=tpu128.config \
--trained_checkpoint_prefix=gs://staging.dreamer8090.appspot.com/data/tpu128/model.ckpt-50000 \
--output_directory=gs://staging.dreamer8090.appspot.com/outmodel/tpu128 \
--add_postprocessing_op=true

# 转换没有量化的tflite

In [0]:
# !tflite_convert \
#   --graph_def_file=gs://mytf9133.appspot.com/train/output/tflite_graph.pb \
#   --input_format=TENSORFLOW_GRAPHDEF \
#   --output_format=TFLITE \
#   --output_file=gs://mytf9133.appspot.com/train/output/tflite_graph.tflite \
#   --inference_type=FLOAT \
#   --input_shapes=1,128,128,3 \
#   --input_arrays=normalized_input_image_tensor \
#   --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
#   --allow_custom_ops \

# 转换量化的tflite

In [30]:
!tflite_convert \
  --graph_def_file=/content/outmodel/tpu128/tflite_graph.pb \
  --output_format=TFLITE \
  --output_file=/content/outmodel/tpu128/tflite_graph.tflite \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --input_shapes=1,128,128,3 \
  --inference_type=QUANTIZED_UINT8 \
  --mean_value=128 \
  --std_dev_values=128 \
  --default_ranges_min=0 \
  --default_ranges_max=6 \
  --change_concat_input_ranges=false \
  --allow_custom_ops
 

2019-10-31 03:36:18.691603: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-31 03:36:18.691913: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559c1229b640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-31 03:36:18.691953: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [0]:
%cd /content 

# 下载模型到mytf4930工程 解压copy到gs://mytf_storage/data/

In [0]:
# cd /tmp
# !curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
# !tar xzf ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz

!gsutil cp -r ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/  gs://lomasky8090_data/data/

In [0]:
import os
os.chdir("object_detection")


In [0]:
# !git clone https://github.com/skyloma/img2.git  
!git clone https://github.com/tensorflow/models.git  
  

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

#This creates a name_pb2.py file from every name.proto file in the \object_detection\protos folder.
!cd models/research; protoc object_detection/protos/*.proto --python_out=.

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python xml_to_csv.py   --image_dir=/content/img2/

In [26]:
%cd /content  
import os
os.chdir("models/research")


/content


In [0]:
!ls

In [0]:
! export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim;python object_detection/generate_tfrecord52.py \
    --csv_input=/content/img2/train_labels.csv \
    --image_dir=/content/img2/train \
    --output_path=/content/train.record  
# !  export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/generate_tfrecord52.py \
#     --csv_input=/content/img2/test_labels.csv \
#     --image_dir=/content/img2/test \
#     --output_path=/content/test.record  